In [18]:
#!/usr/bin/env python

"""
Insert data into HBase with a Python script.
To create the table, first use the hbase shell. We are going to create a
namespace called "sample_data". The table for this script is called "rfic",
as we will be inserting Request for Information Cases from the City of
Indianapolis.
Our table will have only one column family named "data", and we are accepting
all table defaults.
Original data source
https://data.indy.gov/dataset/Request-for-Information-Cases/ts4b-8qa9
% hbase shell
hbase> create_namespace "sample_data"
hbase> create "sample_data:rfic", "data"
"""

#antes de importar happybase se debe instalar
#pip install happybase

import csv
import happybase
import time

batch_size = 1000
host = "192.168.1.108"
#file_path = "Request_for_Information_Cases.csv"
namespace = "calidadaire"
row_count = 0
start_time = time.time()
table_name = "mediciones"


def connect_to_hbase():
    """ Connect to HBase server.
    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    conn = happybase.Connection(host = host,
        table_prefix = namespace,
        table_prefix_separator = ":")
    conn.open()
    table = conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    
    
    return conn, batch

def insert_row(batch, row):
    """ Insert a row into HBase.
    Write the row to the batch. When the batch size is reached, rows will be
    sent to the database.
    Rows have the following schema:
        [ id, keyword, subcategory, type, township, city, zip, council_district,
          opened, closed, status, origin, location ]
    """
   
    batch.put(row[0], { "trafico:totalvehiculostunel": row[1], "trafico:totalvehiculoscalle30": row[2],
        "trafico:velocidadmediasuperficie": row[3], "trafico:velocidadmediatunel": row[4]}, "20160612")


#def read_csv():
    #csvfile = open(file_path, "r")
    #csvreader = csv.reader(csvfile)
    #return csvreader, csvfile


# After everything has been defined, run the script.
conn, batch = connect_to_hbase()
print "Connect to HBase. table name: %s, batch size: %i" % (table_name, batch_size)
#csvreader, csvfile = read_csv()
#print "Connected to file. name: %s" % (file_path)

try:
    
    row=['20160102','1','2','3','0']
    #ejemplo de fila de valores

    insert_row(batch, row)
    
    # Loop through the rows. The first row contains column headers, so skip that
    # row. Insert all remaining rows into the database.
    #for row in csvreader:
    #    row_count += 1
    #    if row_count == 1:
    #        pass
    #    else:
    #        insert_row(batch, row)

    # If there are any leftover rows in the batch, send them now.
    batch.send()
finally:
    # No matter what happens, close the file handle.
#    csvfile.close()
    conn.close()

duration = time.time() - start_time
print "Done. row count: %i, duration: %.3f s" % (row_count, duration)

Connect to HBase. table name: mediciones, batch size: 1000
<type 'list'>
Done. row count: 0, duration: 0.018 s
